In [ ]:
#Adding Packages
import requests as rq
import json
import os
import wikipedia
from urllib.request import urlopen
import pandas as pd
from bs4 import BeautifulSoup
from textblob import TextBlob
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm

In [1]:
#How to get a full WIkipedia article using json ##
url = "https://en.wikipedia.org/w/api.php?format=json&action=query&titles=Hamilton&prop=revisions&rvprop=content"
data = rq.get(url).json()
bfile = open('Hamilton.txt', 'w+')
json.dump(data, bfile)
bfile.close()

/Users/morganhogenmiller/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


## Referencing List of Broadway Shows Since 2010, CORGI ##

In [2]:
Broadway_Original = pd.read_csv('/Users/morganhogenmiller/Desktop/DA 401/Data sets/broadway(corgi).csv') #Read in CORGI data set

In [52]:
Google_Parameters= Broadway_Original[['Name','Year','Month','Day','Type']] #Reduce data set to variables to input into GT

#Filter to shows after 2010 only
Google_Parameters=Google_Parameters.loc[Google_Parameters['Year']>2007] 
Google_Parameters=Google_Parameters.loc[Google_Parameters['Type']=='Musical']


In [57]:
#Provides a list of shows in form needed for function 
show_list=list(Google_Parameters.Name.unique()) 
show_list.sort() #in alphabetical order
show_list

['13',
 '9 To 5',
 'A Catered Affair',
 "A Chorus Line '06",
 'A Christmas Story',
 "A Gentleman'S Guide To Love And Murder",
 'A Little Night Music',
 'A Night With Janis Joplin',
 'A Tale Of Two Cities',
 'After Midnight',
 'Aladdin',
 'All About Me',
 'Allegiance',
 'Amazing Grace',
 'American Idiot',
 'American Psycho',
 'An American In Paris',
 "Annie '12",
 "Anything Goes '11",
 'Avenue Q',
 "Baby It'S You!",
 'Beautiful: The Carole King Musical',
 'Big Fish',
 'Billy Elliot: The Musical',
 'Bloody Bloody Andrew Jackson',
 'Bonnie And Clyde',
 'Bright Star',
 'Bring It On The Musical',
 'Bullets Over Broadway',
 'Burn The Floor',
 'Bye Bye Birdie',
 'Cabaret (musical)',
 'Catch Me If You Can',
 'Cats 2016',
 'Chaplin',
 'Chicago',
 'Cinderella',
 'Come Fly Away',
 'Cry Baby',
 'Curtains',
 'Dames At Sea',
 'Disaster!',
 'Doctor Zhivago',
 "Dr. Seuss' How The Grinch Stole Christmas! 07",
 'Elf',
 "Elf '12",
 'Everyday Rapture',
 'Evita',
 'Fela!',
 'Fiddler On The Roof 2015',
 'Fi

## Counting number of articles within Wikipedia category for Broadway musicals##

In [42]:
#To clean, make something that says if it begins with Category, take it out
def categories(category):
    """This function takes a Wikipedia category as an input and outputs the number of Wikipedia
    articles that exist in that category."""
    count = "0"
    articles = list()

    # Loop through each musical and update count each time
    while True:
        url = "https://en.wikipedia.org/w/api.php?format=json&action=query&list=categorymembers&cmtitle=Category:"+category+"&cmlimit=500&prop=revisions&rvprop=content&cmcontinue="+ count
        data = rq.get(url).json()
        articles.extend(data['query']['categorymembers'])
        try:
            count = data['continue']['cmcontinue']
        except:
            break

    # Get the heroes and villains and ambiguous's names from the data
    articles = [x['title'] for x in articles]

    return("Number of articles in the "+ category + " category: "+ str(len(articles)), articles)

In [47]:
results=categories('Broadway_musicals')
articles=results[1] # creating an object for the list of musical titles
#categories('Tony_Award_Winning_musicals')
len(articles)

884

## Download text data for each show ##

### Before doing the step below, I had to change the names of shows in the original broadway data set to get the correct Wikipedia summeries ###

1. The shows I manusally changed were:
2. Beautiful-> Beautiful: The Carol King Musical
3. Cabaret'14 -> Cabaret (musical)
4. Fela! '14 -> Fela! (musical)
5. Hamilton -> Hamilton (musical)
6. The People In the Picture->The People in the Picture
7. Spring Awakening 2014 -> Spring Awakening


['13',
 '9 To 5',
 'A Catered Affair',
 "A Chorus Line '06",
 'A Christmas Story',
 "A Gentleman'S Guide To Love And Murder",
 'A Little Night Music',
 'A Night With Janis Joplin',
 'A Tale Of Two Cities',
 'After Midnight',
 'Aladdin',
 'All About Me',
 'Allegiance',
 'Amazing Grace',
 'American Idiot',
 'American Psycho',
 'An American In Paris',
 "Annie '12",
 "Anything Goes '11",
 'Avenue Q',
 "Baby It'S You!",
 'Beautiful: The Carole King Musical',
 'Big Fish',
 'Billy Elliot: The Musical',
 'Bloody Bloody Andrew Jackson',
 'Bonnie And Clyde',
 'Bright Star',
 'Bring It On The Musical',
 'Bullets Over Broadway',
 'Burn The Floor',
 'Bye Bye Birdie',
 'Cabaret (musical)',
 'Catch Me If You Can',
 'Cats 2016',
 'Chaplin',
 'Chicago',
 'Cinderella',
 'Come Fly Away',
 'Cry Baby',
 'Curtains',
 'Dames At Sea',
 'Disaster!',
 'Doctor Zhivago',
 "Dr. Seuss' How The Grinch Stole Christmas! 07",
 'Elf',
 "Elf '12",
 'Everyday Rapture',
 'Evita',
 'Fela!',
 'Fiddler On The Roof 2015',
 'Fi

In [56]:
#Making a list of the summaries of all 113 shows
reviews=[]
show_list2=[]
for a in show_list[1:65]:
    print(a)
    if '(musical)' in a: #if already states in name that it is musical, add
        show_list2.append(a)
        w=wikipedia.page(a)
        reviews.append(w.summary)
    elif 'Musical' in a: # "  " ^
        show_list2.append(a)
        w=wikipedia.page(a)
        reviews.append(w.summary)
    elif '(musical)' not in a: #special case show...name in data set not akin to what Wikipedia needs for search
        if a == 'Paramour':
            show_list2.append(a)
            w=wikipedia.page(a +'(Cirque du Soleil)')
            reviews.append(w.summary)
        if a in ['Fela!','Godspell', 'The People in the Picture']: 
            show_list2.append(a)#Shows Wikipedia doesn't identitfy with 'musical' in the name
            w=wikipedia.page(a)
            reviews.append(w.summary)
        if a not in ['Fela!','Godspell','Paramour','The People in the Picture']:  
            show_list2.append(a)
            w=wikipedia.page(a +'(musical)') #make sure shows with book, movie, etc. names pull only Wikipedia's musical article
            reviews.append(w.summary)
        

9 To 5
A Catered Affair
A Chorus Line '06
A Christmas Story
A Gentleman'S Guide To Love And Murder
A Little Night Music
A Night With Janis Joplin
A Tale Of Two Cities
After Midnight
Aladdin
All About Me
Allegiance
Amazing Grace
American Idiot
American Psycho
An American In Paris
Annie '12
Anything Goes '11
Avenue Q
Baby It'S You!
Beautiful: The Carole King Musical
Big Fish
Billy Elliot: The Musical
Bloody Bloody Andrew Jackson
Bonnie And Clyde
Bright Star
Bring It On The Musical
Bullets Over Broadway
Burn The Floor
Bye Bye Birdie
Cabaret (musical)
Catch Me If You Can
Cats 2016
Chaplin
Chicago
Cinderella
Come Fly Away
Cry Baby
Curtains
Dames At Sea
Disaster!
Doctor Zhivago
Dr. Seuss' How The Grinch Stole Christmas! 07
Elf
Elf '12
Everyday Rapture
Evita
Fela!
Fiddler On The Roof 2015
Finding Neverland
Finian'S Rainbow
First Date
Follies '11
Fun Home
Ghost
Gigi
Glory Days
Godspell
Grease 07


PageError: Page id "greater 07 musical" does not match any pages. Try another id!

In [ ]:
#Making sure all shows have reviews
len(reviews)   
list(set(show_list) - set(show_list2))


## Exporting Each Summary to a Folder ##

In [16]:
os.chdir ('/Users/morganhogenmiller/Desktop/GitHub/DA_401/WikiSummaries')

#Making text documeents for each of the reviews
for i,r in enumerate(reviews):
    file=open( 'show' + str(i) + '.txt', 'w')
    file.write(r)
    

## Beginning to tokenize ##
### First by sentences ###

In [39]:
#Sentences
from nltk import sent_tokenize
directory='/Users/morganhogenmiller/Desktop/GitHub/DA_401/WikiSummaries'
def sents(directory): 
    
    """
    Uses the built in sentence tokenizer to extract sentences from the
    paragraphs. Note that this method uses BeautifulSoup to parse HTML.        """     
    for file in os.listdir(directory): 
        with open(os.path.join(directory, file)) as f:
            #for sentence in sent_tokenize(f):                
            print(f)
sents(directory)

<_io.TextIOWrapper name='/Users/morganhogenmiller/Desktop/GitHub/DA_401/WikiSummaries/.DS_Store' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='/Users/morganhogenmiller/Desktop/GitHub/DA_401/WikiSummaries/A Christmas Story.txt' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name="/Users/morganhogenmiller/Desktop/GitHub/DA_401/WikiSummaries/A Gentleman'S Guide To Love And Murder.txt" mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='/Users/morganhogenmiller/Desktop/GitHub/DA_401/WikiSummaries/A Little Night Music.txt' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='/Users/morganhogenmiller/Desktop/GitHub/DA_401/WikiSummaries/A Night With Janis Joplin.txt' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='/Users/morganhogenmiller/Desktop/GitHub/DA_401/WikiSummaries/After Midnight.txt' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='/Users/morganhogenmiller/Desktop/GitHub/DA_401/WikiSummaries/Aladdin.txt' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='/Users/morganhog

In [ ]:
#another example of tokenization from chapter 4
import nltk
import string

def tokenize(text):
    stem = nltk.stem.SnowballStemmer('english')
    text = text.lower()

    for token in nltk.word_tokenize(text):
        if token in string.punctuation: continue
        yield stem.stem(token)

corpus = [
    "The elephant sneezed at the sight of potatoes.",
    "Bats can see via echolocation. See the bat sight sneeze!",
    "Wondering, she opened the door to the studio.",
]

In [ ]:
#within sentences, words

### Vectorization ##

In [ ]:
#one-hot encoding, better for small documents because of non-repeatability 
def vectorize(doc):
    return {
        token: True
        for token in doc
    }

vectors = map(vectorize, corpus)

In [ ]:
#TF-IDF, using the TextCollection class
#Probably best to use
from nltk.text import TextCollection

def vectorize(corpus):
    corpus = [tokenize(doc) for doc in corpus]
    texts  = TextCollection(corpus)

    for doc in corpus:
        yield {
            term: texts.tf_idf(term, doc)
            for term in doc
        }

## DO some type of normalization ##
lemmatization
stemming
removing words that occur < 10% of the time
removing stop words

## Initializing the Model ##

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

def identity(words):
    return words

def create_pipeline(estimator, reduction=False):

    steps = [
        ('normalize', TextNormalizer()),
        ('vectorize', TfidfVectorizer(
            tokenizer=identity, preprocessor=None, lowercase=False
        ))
    ]

    if reduction:
        steps.append((
            'reduction', TruncatedSVD(n_components=10000)
        ))

    # Add the estimator
    steps.append(('classifier', estimator))
    return Pipeline(steps)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier

models = []
for form in (LogisticRegression, MultinomialNB, SGDClassifier):
    models.append(create_pipeline(form(), True))
    models.append(create_pipeline(form(), False))

In [ ]:
for model in models:
    model.fit(train_docs, train_labels)

## Set up for splits ##

In [46]:
#Assessing what happens if we use TextBlob, a pretrained sentiment classifier...for reference
summary_sent=[]
for r in reviews:
    blob = TextBlob(r) #grabs the text in each headline
    sent=blob.sentiment.polarity #polarity between (0-1) of the
    summary_sent.append(sent)
summary_sent

summary_sentimentality=pd.DataFrame({'Name':show_list ,'Sentimentality':summary_sent}) #Makes dataframe of 

summary_sentimentality.sort_values(by=['Sentimentality'])

,Name,Sentimentality
17,Bloody Bloody Andrew Jackson,-0.320000
4,After Midnight,-0.104167
20,Bring It On The Musical,-0.071441
45,Hands On A Hardbody,-0.066667
37,First Date,-0.062500
15,Big Fish,-0.042857
5,Aladdin,-0.022203
6,Allegiance,-0.020833
10,An American In Paris,-0.012500
47,Holler If Ya Hear Me,0.000000


## Analysis ##

###  Distribution of Sentiment Scores ###

In [ ]:
#Exporting sentimentalities as a csv file
pd.DataFrame.to_csv(summary_sentimentality,'summary_sentimentality.csv')

In [ ]:
#Distribution of Wikipedia Sentimentalities
plot=summary_sentimentality['Sentimentality'].hist()
plot.set_xlabel("Sentimentality")
plot.set_ylabel("Frequency")
plot.set_title("Distribution of Wikipedia Summary Sentimentalities")

plt.show()

In [ ]:
summary_sentimentality.head(10) #lowest sentimentalities
summary_sentimentality.tail(10) #top sentimentalities

#Number of weeks each show was on Broadway
weeks=pd.DataFrame(Google_Parameters['Name'].value_counts())
weeks=weeks.reset_index()
weeks=weeks.rename(index=str, columns={"index": "Name1", "Name": "Weeks"})
weeks=weeks.sort_values(by=['Name1'])
weeks=weeks.reset_index()

#Merging sentimentality and weeks on Broadway 
Wikipedia = pd.concat([weeks, summary_sentimentality], axis=1)
Wikipedia.drop(['index','Name1'],axis=1) #Clean
Wikipedia = Wikipedia[['Name', 'Weeks', 'Sentimentality']]

pd.DataFrame.to_csv(Wikipedia,'Wikipedia_Data.csv')

### Weeks v. Sentimentality ###

In [ ]:
#log transform weeks 
Wikipedia['Weeks'] = np.log(Wikipedia['Weeks'])
Wikipedia['Weeks'].hist()
plt.show()


In [ ]:
#Scatter plot
Wikipedia.plot.scatter(x='Weeks', y='Sentimentality', rot=0)
plt.show()

## Insert a question of interest here ###